<a href="https://colab.research.google.com/github/KUHELI08/gen_ai_data_preprocessing_and_data_cleaning/blob/main/Mistral_7b%2Cchroma_db_multi_doc_retriever%2Clangchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Thu Jan 16 05:26:42 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.9 MB/s eta 0:00:

In [6]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00


In [7]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [8]:
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip

--2025-01-16 05:34:45--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2025-01-16 05:34:45--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2025-01-16 05:34:45 (69.6 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638

In [9]:
# Initializes a DirectoryLoader to load text files from the specified directory.
loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)

# Loads all the documents present in the directory.
documents = loader.load()

In [10]:
documents

[Document(metadata={'source': 'stock_market_june_2023/Kiplin Metals Inc.txt'}, page_content='The global shift towards electric vehicles (EVs) is driving an exponential increase in electricity demand. Countries worldwide, including oil-producing nations, are turning to nuclear energy to meet this growing need while transitioning away from fossil fuels. Consequently, the demand for uranium, essential for fueling the nuclear power sector, has experienced a significant surge in the past year after decades of stagnation, clearly showcased in the uranium spot price.\n\nEveryone is drilling for uranium, and companies like Kiplin Metals Inc. (TSXV: KIP | FWB: 17G1 | OTC: ALDVF), which are strategically located in uranium-rich regions like Saskatchewan, Canada, stand to gain the most.\nKiplin’s mining location next to Cluff Lake in Saskatchewan is a known vein of uranium that attracted companies like Urano in the past. The area has historically produced approximately 62,000,000 lbs of yellowcak

In [11]:
# Initializes a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Splits the loaded documents into chunks of text using the defined text_splitter.
texts = text_splitter.split_documents(documents)

In [12]:
len(texts)


49

In [13]:
texts[2]

Document(metadata={'source': 'stock_market_june_2023/Kiplin Metals Inc.txt'}, page_content='Kiplin’s mining location next to Cluff Lake in Saskatchewan is a known vein of uranium that attracted companies like Urano in the past. The area has historically produced approximately 62,000,000 lbs of yellowcake uranium, making it a competitive place for other companies to buy claims.')

In [25]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

In [31]:
notebook_login()

In [34]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read)

In [35]:

model = "mistralai/Mistral-7B-Instruct-v0.1"




In [39]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch




In [41]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map="auto",
                                             torch_dtype=torch.float16,
                                             load_in_8bit=True,
                                             trust_remote_code=True,
                                             use_auth_token=True,
                                            )

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [49]:
# Create the text generation pipeline
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

Device set to use cuda:0


In [50]:
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

In [51]:
from transformers import pipeline

In [52]:
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

model_name = "sentence-transformers/all-mpnet-base-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)

In [53]:
# Sets the directory where the embeddings will be stored.
persist_directory = 'db'

# Sets the HuggingFaceEmbeddings object as the embedding to use.
embedding = hf

# Uses the Chroma module to convert the texts into embeddings using the specified HuggingFace embeddings.
# The resulting embeddings are stored in the persist_directory.
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=hf,
                                 persist_directory=persist_directory)


In [54]:
# Persists the generated embeddings database to disk.
vectordb.persist()

# Releases the memory held by the vectordb object by setting it to None.
vectordb = None

 #Re-initializes the Chroma object from the persisted directory with the specified HuggingFace embeddings.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=hf)


In [55]:
# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# create the chain to answer questions
# Initializes a RetrievalQA object with the specified HuggingFacePipeline (llm), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)


 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])

In [56]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Arbor Metals Corp.
Tesla Inc.
Kiplin Metals Inc.
Microsoft
NVIDIA
Shopify
Intuitive Surgical
MercadoLibre
Growth stocks like these are ideal for investors seeking short-term gains and long-term resilience. These companies are primed to withstand future market fluctuations through innovation, market opportunities, embedding in recession-resistant sectors, and committing to diversifying their portfolios.

Arbor Metals Corp.
Tesla Inc.
Kiplin Metals Inc.
Microsoft
NVIDIA
Shopify
Intuitive Surgical
MercadoLibre
Growth stocks like these are ideal for investors seeking short-term gains and long-term resilience. These companies are primed to withstand future market fluctuations through innovation, market opportunities, embedding in recession-resistant sectors, and committing to diversifying their portfolios.

Institutional ETFs ass

In [57]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
llm_response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'query': 'How much has Microsoft invested in OpenAI?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nLet’s focus on Microsoft’s artificial intelligence (AI) prospects. As AI propels itself into the mainstream, it’s not surprising to see investors eager to fortify their portfolios with AI stocks. Microsoft’s recent $1 billion investment in OpenAI – the creator of ChatGPT and the AI image generator Dall-E- positions the stock as a prime candidate for investment.\n\nLet’s focus on Microsoft’s artificial intelligence (AI) prospects. As AI propels itself into the mainstream, it’s not surprising to see investors eager to fortify their portfolios with AI stocks. Microsoft’s recent $1 billion investment in OpenAI – the creator of ChatGPT and the AI image generator Dall-E- positions the stock as a prime candidate for investment.\n\nMicrosoft’s strategic move to in

In [58]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.

To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.

In the seven years since its initial public offering, Shopify has been profitable in five, a commendable tra

In [60]:
query = "What the nvidia corporation do"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Nvidia’s shares have seen a massive surge since the beginning of 2023, indicating that the market recognizes the company’s substantial potential in the AI industry. While some may consider the stock to be pricy, Nvidia has a lot of things going for it to help justify the premium price tag.

Nvidia’s shares have seen a massive surge since the beginning of 2023, indicating that the market recognizes the company’s substantial potential in the AI industry. While some may consider the stock to be pricy, Nvidia has a lot of things going for it to help justify the premium price tag.

Market analysts anticipate the AI sector to grow twentyfold, reaching nearly $2 trillion by the close of 2030 as the technology pervades domains from supply chains to mobile applications. If you equate this opportunity to the oil boom, Nvidia isn’t dri